<a href="https://colab.research.google.com/github/LemurPwned/filmweb-nlp/blob/master/Transformers_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers

     |████████████████████████████████| 317kB 3.4MB/s 
     |████████████████████████████████| 645kB 43.6MB/s 
     |████████████████████████████████| 860kB 33.0MB/s 
     |████████████████████████████████| 1.0MB 36.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=65f03410567e4327f04db97c3063efa45f1c77bac779f7f45957981926762ce9
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses


In [5]:
import pandas as pd 
from transformers import BertForSequenceClassification,BertTokenizer
model_name = 'bert-base-multilingual-cased'

df = pd.read_csv('/content/reviews_for_bert.csv')
labels = df['rating'].unique().tolist()
if 0 not in labels:
  # labels must start from 0
  labels = [label - 1 for label in labels]

tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name,
                                                      num_labels=len(labels))

INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-vocab.txt from cache at /root/.cache/torch/transformers/96435fa287fbf7e469185f1062386e05a075cadbf6838b74da22bf64b080bc32.99bcd55fc66f4f3360bc49ba472b940b8dcf223ea6a345deb969d607ca900729
INFO:transformers.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-config.json not found in cache or force_download set to True, downloading to /tmp/tmpeu096e88
100%|██████████| 521/521 [00:00<00:00, 174930.55B/s]
INFO:transformers.file_utils:copying /tmp/tmpeu096e88 to cache at /root/.cache/torch/transformers/45629519f3117b89d89fd9c740073d8e4c1f0a70f9842476185100a8afe715d1.83b0fa3d7f1ac0e113ad300189a938c6f14d0588a4200f30eef109d0a047c484
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/transformers/45629519f3117b89d89fd9c740073d8e4c1f0a70f9842476185100a8afe715d1.83b0fa3d7f1ac0e113ad300189a938c6f14d0588a4200

In [109]:
from transformers.data.processors.utils import InputExample
from transformers.data.processors.glue import glue_convert_examples_to_features
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
import torch.utils.data as data_utils
import numpy as np 

BATCH_SIZE = 32
MAX_SEQ_LEN = 128


def select_field(features, field):
  return [
        [
            feature[field]
        ]
        for feature in features
    ]
  
def prepare_dataset(df):
  df_sampl = df
  sents = df_sampl['content'].values
  labels = df_sampl['rating'].astype(int).values
  labels_unique = df['rating'].astype(int).unique()
  # print(labels_unique)
  # print(labels)
  # label_map = {i: i for i in range(11)}
  guid =0 
  input_examples = []
  for sent, label in zip(sents, labels):
    input_examples.append(InputExample(
        guid, text_a=sent, text_b=None, label=label
    ))
  tokenizer = BertTokenizer.from_pretrained(model_name)
  features = glue_convert_examples_to_features(
      input_examples,
      tokenizer=tokenizer,
      max_length=MAX_SEQ_LEN,
      output_mode='classification',
      label_list = labels_unique
  )


  all_input_ids = torch.tensor([feature.input_ids for feature in features], dtype=torch.long)
  all_input_mask = torch.tensor([feature.attention_mask for feature in features], dtype=torch.long)
  all_segment_ids = torch.tensor([feature.token_type_ids for feature in features], dtype=torch.long)
  all_label_ids = torch.tensor([feature.label for feature in features], dtype=torch.long)

  dataset = TensorDataset(all_input_ids, 
                          all_input_mask, 
                          all_segment_ids, 
                          all_label_ids)
  return dataset 

msk = np.random.rand(len(df)) < 0.8
train = df[msk]
test = df[~msk]
train_datatset = prepare_dataset(train)
test_dataset = prepare_dataset(test)
train_dataloader = data_utils.DataLoader(train_datatset, 
                                         batch_size=BATCH_SIZE, 
                                         shuffle=True)
test_dataloader = data_utils.DataLoader(test_dataset, 
                                        batch_size=BATCH_SIZE, 
                                        shuffle=True)

INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-vocab.txt from cache at /root/.cache/torch/transformers/96435fa287fbf7e469185f1062386e05a075cadbf6838b74da22bf64b080bc32.99bcd55fc66f4f3360bc49ba472b940b8dcf223ea6a345deb969d607ca900729
INFO:transformers.data.processors.glue:Writing example 0
INFO:transformers.data.processors.glue:*** Example ***
INFO:transformers.data.processors.glue:guid: 0
INFO:transformers.data.processors.glue:input_ids: 101 10190 42185 10157 14552 10418 11033 18775 100711 14950 10104 11710 33504 117 76206 68782 16600 73264 10280 83366 10692 177 11342 17894 10797 53873 31519 11335 10132 45840 12354 78098 50545 15088 41152 31167 40574 17249 20149 28978 10147 14909 10113 14042 187 14660 10162 25651 19648 24831 10174 119 10685 61048 20868 10963 27966 88629 60914 73837 187 13342 22555 27302 175 81349 88077 10644 87468 52214 20157 23462 10219 11133 12518 39860 38356 44762 10113 10149 28780 9

In [0]:
from transformers import AdamW, WarmupLinearSchedule

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

lr = 2e-5
warmup=0.1

max_grad_norm = 1.0
num_total_steps = 1000
num_warmup_steps = 100
warmup_proportion = float(num_warmup_steps) / float(num_total_steps)  # 0.1

optimizer = AdamW(model.parameters(), lr=lr, correct_bias=False)  # To reproduce BertAdam specific behavior set correct_bias=False
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=num_warmup_steps, t_total=num_total_steps)  # PyTorch scheduler

In [112]:
from tqdm import trange 
### and used like this:
# for batch in train_data:
#     loss = model(batch)
#     loss.backward()
#     torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)  # Gradient clipping is not in AdamW anymore (so you can use amp without issue)
#     optimizer.step()
#     scheduler.step()
#     optimizer.zero_grad()
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

EPOCHS_NUM = 4
loss_history = []

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)
model = BertForSequenceClassification.from_pretrained(model_name,
                                                      num_labels=len(labels))
model.cuda()

for epoch in trange(EPOCHS_NUM, desc='EPOCH'):
  # set to training mode
  model.train()
  total_loss =0
  for step, batch in enumerate(train_dataloader):
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_segment_ids, b_labels = batch
    outputs = model(b_input_ids, 
                    token_type_ids=None,
                    attention_mask=b_input_mask,
                    labels=b_labels)
    loss = outputs[0]
    loss_history.append(loss.item())
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)  # Gradient clipping is not in AdamW anymore (so you can use amp without issue)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
    total_loss += loss.item()
  print(f"Total loss is {total_loss}")

  model.eval()
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps = 0
  for batch in test_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_segment_ids, b_labels = batch
    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      logits = model(b_input_ids, 
                     token_type_ids=None, 
                     attention_mask=b_input_mask)[0]
    
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

  print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-config.json from cache at /root/.cache/torch/transformers/45629519f3117b89d89fd9c740073d8e4c1f0a70f9842476185100a8afe715d1.83b0fa3d7f1ac0e113ad300189a938c6f14d0588a4200f30eef109d0a047c484
INFO:transformers.configuration_utils:Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 10,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",

Total loss is 36.77368211746216




EPOCH:  25%|██▌       | 1/4 [00:23<01:11, 23.71s/it]

Validation Accuracy: 0.30568181818181817
Total loss is 36.796571493148804




EPOCH:  50%|█████     | 2/4 [00:47<00:47, 23.69s/it]

Validation Accuracy: 0.3176136363636364
Total loss is 36.829204082489014




EPOCH:  75%|███████▌  | 3/4 [01:11<00:23, 23.68s/it]

Validation Accuracy: 0.3534090909090909
Total loss is 36.76258611679077




EPOCH: 100%|██████████| 4/4 [01:34<00:00, 23.68s/it]



Validation Accuracy: 0.34147727272727274
